# Agregagador de CSV's

Agrega os csv's de cada órgão em um só csv (remuneracao_servidores.csv), criando os atributos 'ORGAO', 'ANO' e 'MES'. 

### Imports

In [1]:
import pandas as pd
import os
import re

## Funções

### Lista Diretórios

In [2]:
def lista_dirs(path):
    dirs=[]
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for direc in d:
            if '.txt' not in direc:
                dirs.append(os.path.join(r, direc))
    return dirs

### Lista os arquivos de um diretório

In [3]:
def lista_arqs(path):
    files = []
    # r = raiz, d = diretorios, f = arquivos
    for r, d, f in os.walk(path):
        for file in f:
            if '.csv' in file:
                f_out = os.path.join(r, file)
                files.append(f_out)
    return files

### Cria o dataframe de cada órgão

In [4]:
def cria_df_orgao(path):
    files = lista_arqs(path)
    df_orgao = pd.DataFrame()
    for f in files:
        if '.csv' in f:
            df_mes = pd.read_csv(f, sep=';', header=0, decimal=',', engine='python')
            
            file_name = f.split("_")[1]
            orgao_name = path.split("/")[2]
            
            mes = file_name[4:6]
            ano = file_name[0:4]
            
            df_mes["ANO"] = ano
            df_mes["MES"] = mes
            
            df_orgao= pd.concat([df_orgao, df_mes], ignore_index=True)
            df_orgao["ORGAO"] = orgao_name
    try:
        df_orgao.drop(['Unnamed: 10'], axis=1, inplace=True)
    except:
        print('Sem Coluna Unnamed')
    return df_orgao

### Cria o dataframe geral

In [5]:
def cria_df_geral(path):
    df_geral = pd.DataFrame()
    dirs = lista_dirs(path)
    for d in dirs:
        if (str(d) == "../csv/PM-ATIVOS" or str(d) == "../csv/FUNDAÇÃO VILA OLIMPICA" or str(d) == "../csv/PRODAM"):
            continue
        print("Processando diretório:", str(d))
        nome_dir = d.split("/")[1]
        ds_orgao = pd.DataFrame()
        df_orgao = cria_df_orgao(d)
        df_geral = pd.concat([df_geral,df_orgao], ignore_index=True)
        df_geral = trata_nan(df_geral)
    return df_geral

### Trata os valores NaN

In [6]:
def trata_nan(df):
    for col in df.columns:
        if(df[col].dtype == 'object'):
            df[col] = df[col].fillna('-')
        if(df[col].dtype == 'float64'):
            df[col] = df[col].fillna(0.00)
    
    return df

## Main

In [8]:
path = '../csv/'
df_geral = cria_df_geral(path)
df_geral.to_csv(r'../ds/remuneracao_servidores.csv')
df_geral

Processando diretório: ../csv/SNPH


/home/vini/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Processando diretório: ../csv/SEPLANCTI
Processando diretório: ../csv/PGE
Processando diretório: ../csv/CGL
Processando diretório: ../csv/SUHAB
Processando diretório: ../csv/FAPEAM
Processando diretório: ../csv/FHEMOAM
Processando diretório: ../csv/SEAD
Processando diretório: ../csv/SERGB
Sem Coluna Unnamed
Processando diretório: ../csv/POLICIA CIVIL
Processando diretório: ../csv/SEC
Processando diretório: ../csv/IMPRENSA OFICIAL
Processando diretório: ../csv/ADAF
Processando diretório: ../csv/IDAM
Processando diretório: ../csv/SEINFRA
Processando diretório: ../csv/FHAJ
Processando diretório: ../csv/SETRAB
Processando diretório: ../csv/SEAD-PENSÃO HANSENIANOS
Processando diretório: ../csv/SRMM
Processando diretório: ../csv/DETRAN
Processando diretório: ../csv/IPAAM
Processando diretório: ../csv/ALFREDO DA MATA
Processando diretório: ../csv/SECOM
Processando diretório: ../csv/AMAZONPREV
Processando diretório: ../csv/CASA CIVIL DO GOVERNO
Processando diretório: ../csv/SEIND
Sem Coluna Un

,REMUNERACAO LEGAL DEVIDA(R$),ANO,CARGO,DESC.TETO(R$),DESCONTOS LEGAIS(R$),FUNCAO,LIQUIDO DISPONIVEL(R$),LOTACAO,MES,NOME,ORGAO,REMUNERACAO LEGAL TOTAL(R$),VINCULO
0,1486.20,2017,AGENTE AQUAVIARIO IV,0.0,152.07,-,1334.13,DIOPE-DIVISAO OPERACIONAL,10,ADEILDO COSTA FARIAS,SNPH,1486.20,ESTATUTARIO
1,1349.42,2017,AGENTE PORTUARIO II,0.0,148.44,AG.PORTUARIO II,1200.98,ITACOATIARA,10,ADONIS BRASIL,SNPH,1349.42,ESTATUTARIO
2,3057.11,2017,AGENTE AQUAVIARIO III,0.0,219.00,-,2838.11,DIOPE-DIVISAO OPERACIONAL,10,AFRANIO SANTOS DA SILVA,SNPH,3057.11,ESTATUTARIO
3,2554.96,2017,AGENTE AQUAVIARIO II,0.0,278.73,-,2276.23,DIOPE-DIVISAO OPERACIONAL,10,ALDEMIR LIMA DE SOUSA,SNPH,2554.96,ESTATUTARIO
4,1624.62,2017,AGENTE PORTUARIO III,0.0,98.96,-,1525.66,DIOPE-DIVISAO OPERACIONAL,10,ALDER JOSE SIMOES DE ALBUQUERQUE,SNPH,1624.62,ESTATUTARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018808,8048.20,2018,PESQUISADOR ASS.-PEA-P.S.N.S.-D,0.0,1836.74,-,6211.46,ESTATUTARIO,06,WORNEI SILVA MIRANDA BRAGA,FMT-AM,8048.20,ESTATUTARIO
2018809,14796.79,2018,FARMAC.BIOQ. -FAB-P.S.N.S.-A,0.0,3584.83,FARMAC BIOQUIMICO,11211.96,ESTATUTARIO,06,WUELTON MARCELO MONTEIRO,FMT-AM,14796.79,ESTATUTARIO
2018810,1945.14,2018,AUXILIAR DE SERV GERAIS - LIMPEZA,0.0,167.77,-,1777.37,ESTATUTARIO,06,YOLANDA FREITAS NOGUTH,FMT-AM,1945.14,ESTATUTARIO
2018811,12361.44,2018,FARMAC.BIOQ. -FAB-P.S.N.S.-D,0.0,2880.83,GERENTE,9480.61,ESTATUTARIO,06,YONNE FRANCIS CHEHUAN MELO,FMT-AM,12361.44,ESTATUTARIO
